In [ ]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])

    #check all was fine
    print(lines[-2:])
    print(str_lines[-2:])
    return str_lines

elf_data = readin('Day11input.txt')